# Advent of Code 2021: Day 6
link to puzzle [here](https://adventofcode.com/2021/day/6)

Imports

In [42]:
from dataclasses import dataclass
from typing import List

!ln -sf ../utils.py .
import utils

Load puzzle input data

In [43]:
DATA_DIR = '../data'
DAY = 6
data = utils.get_puzzle_input(day=DAY, input_dir=DATA_DIR)

Classes

In [44]:
@dataclass
class LanternfishPop:
    """model lanternfish population from an initial state of internal timer values. 
    
    Lanternfish populations are modeled as counts of 
        * parent lanternfish X days from giving birth to child lanternsifh, and
        * child lanternfish Y days from maturing to parent lantern fish
    where 
        * 0 <= X < `time_to_birth` (X an integer) 
        * 0 <= Y < `time_to_mature` (Y an integer)
    
    Lanternfish populations start from an initial state defining the parent lanternfish population,
    and no children lanternfish. We represent the parent populations as arrays whose value
    at some position `i` is equal to the number of parent lanternfish whose internal timer value is
    currently `i`. Children lanternfish populations are represented similarly, the only difference being
    that the length of parent and child lanternfish population arrays are `time_to_birth` and `time_to_mature`,
    repsectively. As such, at initialization, the population arrays look as follows.
        * parent population stored in `parent_pop` and `parent_pop[i]` is equal to the number
          of times `i` appears in `initial_state`, `0 <= i < time_to_birth`
        * child population stored in `child_pop` and `child_pop[j] = 0` for all `0 < j <= time_to_mature`
    
    As days elapse, the child and parent lanternfish populations change according to the following rules.
    
    """
    # initial state of lanternfish population given as list of integers where each item
    # of the list represents an individual parent lanternfish's internal timer value
    initial_state: List[int]
    
    # length of birthing cycle for parent lanternfish
    time_to_birth: int = 7
        
    # time it takes child lanternfish to mature to a parent lanternfish, where
    # the difference between a child and parent is the length of the birthing cycle
    time_to_mature: int = 9
        
    def __post_init__(self):
        self._initialize_child_pop()
        self._initialize_parent_pop()
        
    def _initialize_child_pop(self):
        """child population always starts out with 0 children
        """
        self.child_pop = [0] * self.time_to_mature
    
    def _initialize_parent_pop(self):
        """starting parent population is defined by `initial_state`
        """
        self.parent_pop = [sum(s == i for s in self.initial_state) for i in range(self.time_to_birth)]
        
    def elapse_day(self):
        """elapsing a day reduces internal timer values by 1 for all lanternfish, so we
        update our population counts by moving counts from i to i - 1 taken modulo `time_to_birth`
        for `parent_pop` and `time_to_mature` for `child_pop`
        """
        self.parent_pop = self.roll(self.parent_pop, -1)
        self.child_pop = self.roll(self.child_pop, -1)
        
    def parents_are_ready_to_birth(self):
        return self.parent_pop[-1] > 0
    
    def children_are_ready_to_mature(self):
        return self.child_pop[-1] > 0
    
    def give_birth(self):
        self.child_pop[-1] += self.parent_pop[-1]
        
    def mature_children(self):
        """when children mature they become parents, so update the parent
        population to include the number of children maturing to adult and reduce
        the child population by the number of fish which matured to parents
        """
        self.parent_pop[0] += self.child_pop[0]
        self.child_pop[0] = 0
        
    def simulate_population_growth(self, days):
        """simulate lanternfish population growth over a specified number of days, `days`
        """
        for day in range(days):
            self.elapse_day()
            self.mature_children()
            self.give_birth()
                
    def reset_population(self):
        """reset lanternfish population back to initial state
        """
        self._initialize_child_pop()
        self._initialize_parent_pop()
        
    def get_current_population_size(self):
        """print the total population of lanternfish at current state
        """
        return sum(self.child_pop) + sum(self.parent_pop)
    
    @staticmethod
    def roll(li, shift):
        """if `li` is a list with `len(li) = n`, then "roll" elements of `li` forward or backward by
        `shift` units, that is map position i -> position (i - shift) mod n. E.g.,
        ```
        >>> li = [1, 2, 3, 4]
        >>> roll(li, 1)
        [4, 1, 2, 3]
        >>> roll(li, -2)
        [3, 4, 1, 2]
        ```
        """
        return [li[(i - shift) % len(li)] for i in range(len(li))]

Functions

In [45]:
def get_initial_state(data):
    return [int(t) for t in data[0].split(',')]

Test

In [46]:
initial_state = [3, 4, 3, 1, 2]
pop = LanternfishPop(initial_state)
pop.simulate_population_growth(days=18)
print(pop.get_current_population_size())

26


#### Part 1

In [47]:
initial_state = get_initial_state(data)
pop = LanternfishPop(initial_state)
pop.simulate_population_growth(days=80)
print(pop.get_current_population_size())

380758


#### Part 2

In [48]:
pop.reset_population()
pop.simulate_population_growth(days=256)
print(pop.get_current_population_size())

1710623015163
